In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
! wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv

--2022-02-25 22:29:22--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.97.140
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.97.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 752335705 (717M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-01.csv’

fhvhv_tripdata_2021 100%[===================>] 717.48M  47.8MB/s    in 11s     

2022-02-25 22:29:34 (62.7 MB/s) - ‘fhvhv_tripdata_2021-01.csv’ saved [752335705/752335705]



In [3]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [3]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   null|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   null|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   null|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   null|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   null|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [4]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [11]:
!head -n 1000 fhvhv_tripdata_2021-01.csv > head.csv

In [5]:
import pandas as pd
df_pandas = pd.read_csv('head.csv')

In [6]:
df_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   hvfhs_license_num     999 non-null    object 
 1   dispatching_base_num  999 non-null    object 
 2   pickup_datetime       999 non-null    object 
 3   dropoff_datetime      999 non-null    object 
 4   PULocationID          999 non-null    int64  
 5   DOLocationID          999 non-null    int64  
 6   SR_Flag               0 non-null      float64
dtypes: float64(1), int64(2), object(4)
memory usage: 54.8+ KB


In [7]:
spark.createDataFrame(df_pandas).show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|    NaN|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|    NaN|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|    NaN|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|    NaN|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|    NaN|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [8]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [4]:
from pyspark.sql import types

schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [5]:
df = (spark.read 
    .option("header", "true")
    .schema(schema)
    .csv('fhvhv_tripdata_2021-01.csv')   
)

In [11]:
df.head()

Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None)

In [6]:
df = df.repartition(24)

In [ ]:
df.write.parquet('fhvhv/2021/01/', mode='overwrite')

In [2]:
df = spark.read.parquet('fhvhv/2021/01/') # reads in all of the data from the parquet files into one df

In [3]:
df.printSchema() # note the schema is known from the parquet

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



If we want to select a few columns:

In [4]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID')\
    .filter(df.hvfhs_license_num == 'HV0003') \
    .show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-12 19:45:41|2021-01-12 19:53:08|          90|         114|
|2021-01-12 19:17:09|2021-01-12 19:30:38|         247|          94|
|2021-01-12 19:12:01|2021-01-12 19:18:34|          61|          35|
|2021-01-12 19:27:58|2021-01-12 19:39:28|          35|          35|
|2021-01-12 19:41:17|2021-01-12 19:44:46|          35|          35|
|2021-01-12 19:49:14|2021-01-12 20:12:36|          35|         217|
|2021-01-12 19:08:26|2021-01-12 19:20:11|          22|          67|
|2021-01-12 19:49:20|2021-01-12 19:56:14|         227|         228|
|2021-01-12 19:14:00|2021-01-12 19:24:38|          90|         170|
|2021-01-12 19:29:00|2021-01-12 19:37:33|         170|          79|
|2021-01-12 19:48:59|2021-01-12 19:56:24|          79|          79|
|2021-01-12 19:17:59|2021-01-12 19:36:33|       

In spark, transformations (selecting columns, joins, group by) are not executed immediately. They are lazy. Spark creates a list of transformations and then once we do an action like `.show()`, `.take()`, `.head()`, `write()`. Then all of the transformations are executed. Actions are "eager". 


In [6]:
from pyspark.sql import functions as F

In [9]:
F.to_date?

To create columns as a transformation using a function use the `.withColumn()` function. You can use user defined functions to accomplish complicated tasks in spark easier than in SQL.

In [13]:
def crazy_function(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    else:
        return f'e/{num:03x}'

crazy_function_udf = F.udf(crazy_function, returnType=types.StringType())

In [20]:
(df
    .withColumn('pickup_date', F.to_date(df.pickup_datetime))
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime))
    .withColumn('base_id', crazy_function_udf(df.dispatching_base_num))
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID', 'base_id')
    .show()
)

+-----------+------------+------------+------------+-------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|base_id|
+-----------+------------+------------+------------+-------+
| 2021-01-12|  2021-01-12|          90|         114|  e/b14|
| 2021-01-12|  2021-01-12|         247|          94|  e/b48|
| 2021-01-12|  2021-01-12|          61|          35|  e/b37|
| 2021-01-12|  2021-01-12|          35|          35|  e/b37|
| 2021-01-12|  2021-01-12|          35|          35|  e/b37|
| 2021-01-12|  2021-01-12|          35|         217|  e/b37|
| 2021-01-12|  2021-01-12|          72|          76|  e/9ce|
| 2021-01-12|  2021-01-12|          77|          76|  e/9ce|
| 2021-01-12|  2021-01-12|          22|          67|  e/b38|
| 2021-01-12|  2021-01-12|         227|         228|  e/b38|
| 2021-01-12|  2021-01-12|          90|         170|  e/b37|
| 2021-01-12|  2021-01-12|         170|          79|  e/b37|
| 2021-01-12|  2021-01-12|          79|          79|  e/b37|
| 2021-01-12|  2021-01-1

In [18]:
crazy_function_udf = F.udf(crazy_function, returnType=types.StringType())